In [2]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from tkinter import ttk
import random
import json 


#Menu Screen:
Menu = tk.Tk()
Menu.title("Bootleg Wordle Menu")
Menu.geometry("600x800")

#Welcome Message:
Welcome_Message = tk.Label(Menu, text = "Welcome to the main menu of Bootleg Wordle! \n" 
                                        "Here, you can select a difficulty, and learn how to play.", 
                           font=('Times New Roman',18)
                          )
Welcome_Message.pack()

#Rules:
How_to_play = tk.Label(Menu, text="How to play:")
How_to_play.pack(pady=10)

Rules_Message = tk.Label(Menu, text="Wordle is a word guessing game.\n"
                                   "You have 6 guesses to match a randomly chosen word!\n"
                                   "Each guess must be 5 letters long.\n"
                                   "The color of the tiles will change to show how close\n"
                                   "your guess was to the word.\n"
                                   "If you get stuck, try typing HELP. \n",
                        font=('Times New Roman', 15)
                        )
Rules_Message.pack(pady=10)  
Rules_Message.pack()


#Explain each difficulty setting:
Easy_mode = tk.Label(Menu, text="Easy mode: No timer, guess can be any combo of 5 letters")
Easy_mode.pack()

Med_mode = tk.Label(Menu, text="Medium mode: No timer, guesses must be valid words!")
Med_mode.pack()

Hard_mode = tk.Label(Menu, text="Hard mode: 1 minute timer, guess must be valid words. Good luck!")
Hard_mode.pack()


#Difficulty variable to hold user's selection
difficulty_var = tk.StringVar()


#Prompt user to select a difficulty:
label_difficulty = tk.Label(Menu, text="Please select a Difficulty:", font=('Arial', 18))
label_difficulty.pack(pady=10)


#Create style preset for Radiobuttons, create Radiobuttons with difficulty options:
style = ttk.Style()
style.configure("TRadiobutton", indicatorsize=0)

radio_easy = ttk.Radiobutton(Menu, text="Easy", variable=difficulty_var, value="Easy", style="TRadiobutton", padding=10)
radio_easy.pack()

radio_medium = ttk.Radiobutton(Menu, text="Medium", variable=difficulty_var, value="Medium", style="TRadiobutton", padding=10)
radio_medium.pack()

radio_hard = ttk.Radiobutton(Menu, text="Hard", variable=difficulty_var, value="Hard", style="TRadiobutton", padding=10)
radio_hard.pack()


#Initializes timer variables 
timer_label = None
countdown_seconds = 60
countdown_id = None


#Define function to start the game when given difficulty level selection:
def start_game(difficulty):
    global countdown_seconds, countdown_id, timer_label, root
    
    if difficulty == "Hard":
        try:
            #Check if the root window exists
            #Reference: https://stackoverflow.com/questions/29076086/see-if-tkinter-widget-exists-not-using-winfo-exists
            if not root.winfo_exists():
                return  # Exit if the root window has been destroyed
                
            if timer_label is None:
                countdown_seconds = 60
                timer_label = tk.Label(root, text=f"Time Left: {countdown_seconds} seconds", font=('Arial', 18))
                timer_label.pack()
                countdown_id = root.after(1000, countdown)
                
        except tk.TclError:
            #Handle TclError if the root window has been destroyed
            pass
    Menu.destroy() #Get rid of main menu screen


#Define function to read selected difficulty and start game with that difficulty:
def select_difficulty(difficulty_var):
    global difficulty
    difficulty = difficulty_var.get()
    
    if difficulty:
        start_game(difficulty)
        
    else:
        messagebox.showwarning("Difficulty Selection","Please select a difficulty level.")


#Function to count down the timer:
def countdown():
    global countdown_seconds, countdown_id, timer_label, gylist
    
    #Reference: https://colinch4.github.io/2023-09-06/13-10-15-355782/
    if countdown_seconds > 0:
        countdown_seconds -= 1
        
        if timer_label:
            timer_label.config(text=f"Time Left: {countdown_seconds} seconds")
            
        #Call the countdown function again after 1000 milliseconds (1 second)
        #Reference: https://stackoverflow.com/questions/36402134/tkinter-root-after-cancel
        if countdown_id:
            #Cancel the previous scheduled call
            root.after_cancel(countdown_id)  
        countdown_id = root.after(1000, countdown) 
    #Reference: https://stackoverflow.com/questions/53580507/disable-enable-button-in-tkinter
    if countdown_seconds <= 0:
        lose_Win_Length_label.config(text=f"You Lose! The word was {actualWord}")
        btnSubmit.config(state=DISABLED)
        return


#Create button to start game:
Start_Game = tk.Button(Menu, text="Start Game", command=lambda:select_difficulty(difficulty_var))
Start_Game.pack()

Menu.mainloop()
#End of Menu -- Start of Game screen:

#Update path with your respective path

#Pull word list from files:
file_path = r'/Users/mikeykapotis/FinalProject/words.json'
with open(file_path, 'r') as file:
    data = json.load(file)



#Create function to choose random word from word list:
def Choose_Random_Word():
    RandomWord = random.choice(data).upper()
    return RandomWord
    
#Chooses Random Word:
actualWord = Choose_Random_Word()


#Creates GUI screen with title and dimension:
root = tk.Tk()
root.title("Bootleg Wordle")
root.geometry("600x800")


#Creates textbox in GUI to type guess in
textbox = tk.Text(root, height=2, font=('Arial',18))
textbox.pack(padx=20,pady=5)


#Label asking for guess:
label = tk.Label(root, text='Please Enter a Guess!', font=('Arial',18))
label.pack()


#Function that will be used inside to make the following keyboard work:
def insert_text(letter):
    current_text = textbox.get("1.0", "end-1c")
    new_text = current_text + letter
    textbox.delete("1.0", "end")
    textbox.insert("1.0", new_text)


#Creates button frame to hold keyboard buttons:
buttonframe = tk.Frame(root)
buttonframe.columnconfigure(0, weight=1)
buttonframe.columnconfigure(1, weight=1)
buttonframe.columnconfigure(2, weight=1)

Alphabet = "QWERTYUIOPASDFGHJKLZXCVBNM"
buttons = []
#Places each button in qwerty order in button frame.
#Each button is assigned the "insert_text" command to actually write the letter when clicked:
for index, letter in enumerate(Alphabet):
    btnltr = tk.Button(buttonframe, text=letter, font=('Arial', 18), command=lambda l=letter: insert_text(l))
    if(index <=9):
        btnltr.grid(row=0, column=index, sticky=tk.W + tk.E)
        
    elif(index >=10 and index <=18):
        btnltr.grid(row=1, column=index-10, sticky=tk.W + tk.E)
        
    else:
        btnltr.grid(row=2, column=index-19, sticky=tk.W + tk.E)
    buttons.append(btnltr)
       
buttonframe.pack()


#Creates function for backspace button:
def Backspace():
    textbox.delete("end-2c", "end")


#Creates Delete button:
btnDelete = tk.Button(buttonframe, text='Delete', font=('Arial',18),command=Backspace)
btnDelete.grid(row=2,column=7,columnspan=2,sticky=tk.W+tk.E)
buttonframe.pack()


#Creates function for clearing the text box after a guess is submitted:
def ClearText():
    textbox.delete("1.0", "end")


#Initialize global turn variable
global turn
turn=1


#Makes Frame to layout the Wordle guesses vertically:
GuessLabels = tk.Frame(root)
GuessLabels.columnconfigure(3, weight=1)
GuessLabels.columnconfigure(4, weight=1)
GuessLabels.columnconfigure(5, weight=1)


#Function that is the actual Wordle game:
def Actual_Game():
    
    global turn
    global actualWord
    global timer_label
    global difficulty
    global countdown_seconds
    global timer_label
    global countdown_id
    global GuessLabels
    global gylist
    global root

    
    #Function to check if all entries in a list are all equal
    def check(list):
        return all(i == list[0] for i in list)


    #HEX codes for colors used in Wordle
    Black = '#000000'
    Yellow = '#c8b653'
    Green = '#6ca965'
    
    
    #Pulls the user's guess from the textbox; Makes it capitalalized and removes blank spaces:
    guess = textbox.get("1.0", "end-1c").upper().strip()


    #Sets default colors to black
    gylist = [Black,Black,Black,Black,Black]


    #If difficulty is hard, we need to have timer:
    if difficulty == "Hard" and timer_label is None:
        timer_label = tk.Label(root, text=f"Time Left: {countdown_seconds} seconds", font=('Arial', 18))
        timer_label.pack()

        countdown_id = root.after(1000, countdown)

    
    #If guess length isn't 5, don't run the game and tell the user:
    if len(guess)!=5 and guess!='HELP':
        lose_Win_Length_label.config(text="Only 5 Letter Words!")
    elif guess == 'HELP':
        if turn == 1:
            suggested = Choose_Random_Word()
        else:
            feasible_words=data
            for word in data:
                for ind, color in enumerate(gylist):
                    if (color==Green and guess[ind]!=word[ind]):
                        feasible_words.remove(word)
                    if (color==Yellow and (guess[ind] not in word)):
                        feasible_words.remove(word)
            suggested = random.choice(feasible_words).upper()
        lose_Win_Length_label.config(text=f"Stuck? Try: {suggested}")
        ClearText()
    elif guess.lower() not in data and difficulty != 'Easy': #Valid words only for med/hard
        lose_Win_Length_label.config(text="Invalid Word")
        Clear

    
    #If 5 letter word is entered, play the game!:
    else:
        lose_Win_Length_label.config(text="")
        
        #Find all Green letters:
        if turn <=6:
            for gcNum, guessChar  in enumerate(guess):
                for acNum, acChar in enumerate(actualWord):
                    if (guessChar == acChar and gcNum == acNum):
                        gylist[gcNum] = Green
            
            #Find all Yellow letters
            for acNum, acChar in enumerate(actualWord):
                if (gylist[acNum] != Green):
                     for gcNum, guessChar  in enumerate(guess):
                        # Make sure character has not already been colored
                          if (guessChar == acChar and gylist[gcNum] == Black):
                              gylist[gcNum] = Yellow
                    
            #Once one yellow character has been found corresponding to a specific actual character, break inner-loop
                              break
            for i, letter in enumerate(guess):
                position = Alphabet.index(letter)
                current_color = buttons[position].cget("fg")
                if current_color != Green and current_color != Yellow and gylist[i]!=Black:
                    buttons[position].config(fg = gylist[i])
                elif current_color != Green and current_color != Yellow and gylist[i]==Black:
                    buttons[position].config(fg = '#808080')     
                elif current_color == Yellow and gylist[i]==Green:
                    buttons[position].config(fg = gylist[i])   


            #Creates Label for each letter in guess, colored accordingly
            k=0
            for k in range(0,5):
                GuessLetter = tk.Label(GuessLabels, text= guess[k], font=('Arial',17),bg=gylist[k],width=6,height=3)
                GuessLetter.grid(row=turn, column=k, sticky=tk.W+tk.E, padx=5, pady=5)
            GuessLabels.pack()

            
            #Checks if all letters are color Green (You got the word!):
            if check(gylist) is True and gylist[0]==Green:
                lose_Win_Length_label.config(text=f"You win! The word was {actualWord}")
                btnSubmit.config(state=DISABLED)
                if countdown_id:
                    root.after_cancel(countdown_id)
                

            #Covers the case where all letters are black or yellow on last turn(You lost!):
            elif turn>=6:
                lose_Win_Length_label.config(text=f"You Lose! The word was {actualWord}")
                btnSubmit.config(state=DISABLED)
                if countdown_id:
                    root.after_cancel(countdown_id)

        
            #Moves to next turn
            turn +=1

            
            #Clears the textbox 
            ClearText()
        

#Function to restart the game without rerunning the code:
def restart_game():
    global turn
    global actualWord
    global timer_label
    global difficulty
    global countdown_seconds
    global countdown_id
    global timer_label
    global GuessLabels

    # Cancel the existing countdown
    if countdown_id:
        root.after_cancel(countdown_id)
        countdown_id = None

    # Reset variables, pick a new word
    turn = 1
    actualWord = Choose_Random_Word()
    countdown_seconds = 60
    countdown_id = None

    # Reset colors, labels, etc
    for button in buttons:
        button.config(fg='#000000')  # Reset button colors

    lose_Win_Length_label.config(text="")  # Clear win/lose label

    textbox.delete("1.0", "end")  # Clear textbox

    btnSubmit.config(state=NORMAL)  # Re-enable the submit button

    # Reset the timer label to its initial state
    if timer_label:
        timer_label.config(text=f"Time Left: {countdown_seconds} seconds")

    if GuessLabels:# Clears the tiles from the screen
        for GuessLetter in GuessLabels.winfo_children():
            GuessLetter.destroy()


#Create frame for submit and restart button:
btnfrm2 = tk.Frame(root)
btnfrm2.columnconfigure(0, weight=1)
btnfrm2.columnconfigure(1, weight=1)
btnfrm2.columnconfigure(2, weight=1)


#Creates Submit button that activates "Actual_Game"
btnSubmit = tk.Button(btnfrm2, text ='Guess',font=('Arial',18),command=Actual_Game)
btnSubmit.grid()
btnfrm2.pack()


#Restart button that activates "restart_game"
btnRestart = tk.Button(btnfrm2, text='Restart', font=('Arial', 18), command=restart_game)
btnRestart.grid(row=0, column=1, sticky=tk.W + tk.E)
btnfrm2.pack()


#Creates a Label to tell the user if they win or lose, and to give error if not 5 character guess:
lose_Win_Length_label = tk.Label(root, text= "", font=('Arial',24))
lose_Win_Length_label.pack()


root.mainloop()